In [25]:
import pandas as pd

import matplotlib.pylab as plt
authors = pd.read_csv(r'C:\Users\ketli\Documents\Summer 2021\ORNL\Repo\reposcanner-data\authors.csv')
repolist = pd.read_csv(r'C:\Users\ketli\Documents\Summer 2021\ORNL\Repo\reposcanner-data\repolist.csv')
rid_sum = authors.groupby(by='RID')['commits'].sum()

# total number of lines 'LIN'
authors['LIN'] = authors['adds'] + authors['dels']

In [26]:
#set_dev1k = The set of developers who contributed at least 1,000 lines of code in total.
def core_dev_1k():
    #Grouped by FID as index, .sum() all values for FID
    fid_info = authors.groupby('FID', sort=True, as_index=True).sum()
    set_dev1k = set(fid_info[fid_info['LIN'] >= 1000].index)
    return set_dev1k

In [27]:
def top_contributors(rid):
    repo = authors.loc[authors['RID'] == rid]
    sort_lines = repo.sort_values(by='LIN', ascending=False)
    core = core_dev_repo(sort_lines)
    return core

In [28]:
def core_dev_repo(sort_lines, pct = 0.8):
    total = 0
    ele = 0
    cutoff = sort_lines['LIN'].sum() * pct
   
    # Iterate each element in list
    # and add them in variable total
    while(ele < len(sort_lines)):
        total = total + sort_lines.iloc[ele]['LIN']
        if total >= cutoff:
            break
        ele += 1
    return sort_lines.iloc[:ele + 1]

In [29]:
#set_coredev = The set of developers who were "core" contributors to at least 1 repository.
def core_dev():
    all_core = []
    set_coredev = set()

    for rid in rid_sum.index:
        core = top_contributors(rid)   
        fid = core['FID'].values.tolist()
        all_core.append(fid)
    
    for i in all_core:
        for j in i:
            set_coredev.add(j)
    return set_coredev

In [30]:
repo_project = pd.merge(authors, repolist[['RID','PID']], on='RID', how='left')

In [31]:
#set_crosscontrib = The set of developers who made commits to repositories belonging to more than one ECP project.
def crosscontrib():
    pid_group = repo_project.groupby('PID')['FID'].apply(list)
    dictionary = {}
    for i in pid_group:
        unique_fid = list(dict.fromkeys(i))
        for j in unique_fid:
            dictionary[j] = dictionary.setdefault(j, 0) + 1        
    set_crosscontrib = set(key for key, value in dictionary.items() if value >= 2)
    return set_crosscontrib

In [32]:
print('Cross contributors: ', crosscontrib())

Cross contributors:  {257, 136, 9, 907, 5136, 8337, 784, 919, 3352, 1561, 28, 8350, 928, 933, 42, 811, 174, 7345, 1210, 957, 318, 64, 65, 834, 9286, 2374, 10056, 9163, 9547, 461, 4815, 592, 1105, 82, 4435, 340, 981, 342, 719, 601, 3934, 3809, 867, 2147, 3428, 487, 104, 3690, 2026, 1005, 623, 113, 499, 1012, 374, 7415, 9464, 6012, 10110}


In [33]:
print('Core devs.', core_dev())

Core devs. {513, 6146, 2054, 9, 10763, 15, 9237, 22, 25, 1561, 28, 1054, 8227, 35, 1063, 9255, 8231, 42, 1070, 8753, 1074, 9781, 54, 2103, 569, 571, 5183, 65, 9286, 584, 586, 76, 5710, 593, 82, 7252, 11352, 601, 605, 609, 101, 104, 105, 8298, 8808, 621, 623, 111, 8824, 5759, 133, 136, 138, 8844, 653, 141, 6800, 9878, 160, 675, 168, 169, 685, 7345, 5298, 3761, 183, 1209, 200, 4815, 4816, 213, 9942, 9943, 5846, 6871, 218, 3809, 228, 755, 2820, 4871, 784, 6934, 283, 284, 795, 796, 799, 811, 815, 818, 6962, 12597, 4407, 830, 318, 320, 832, 322, 6469, 328, 10056, 9547, 5964, 845, 6992, 849, 338, 10579, 340, 342, 856, 3418, 859, 6495, 355, 8556, 374, 6012, 6014, 899, 903, 908, 8077, 400, 6551, 4507, 8607, 928, 929, 933, 434, 8628, 6068, 438, 951, 956, 957, 10172, 1983, 11201, 6082, 1990, 969, 461, 973, 467, 981, 2518, 986, 9182, 478, 2022, 487, 1005, 8686, 7663, 5614, 9202, 499, 1012, 1014, 1015, 1017, 1021, 1022}


In [34]:
print('Devs 1k lines: ',  core_dev_1k())

Devs 1k lines:  {513, 2, 1025, 2562, 4612, 2054, 6146, 9, 10763, 8206, 15, 10257, 3092, 22, 1559, 25, 1561, 28, 29, 1054, 9759, 5664, 6178, 35, 8227, 9252, 1063, 552, 3113, 42, 8231, 9255, 1070, 8753, 1074, 9781, 54, 10805, 10807, 57, 569, 571, 62, 5183, 576, 65, 5184, 9286, 3655, 584, 586, 1099, 76, 3148, 8269, 1103, 592, 593, 82, 10830, 7252, 10832, 10833, 10834, 11352, 601, 605, 4701, 1120, 609, 2147, 101, 104, 105, 6760, 8298, 8808, 621, 111, 623, 1651, 6772, 4215, 8824, 124, 1149, 3710, 5245, 5759, 133, 5767, 136, 9353, 138, 8844, 141, 653, 9870, 6800, 8337, 9878, 6811, 8350, 160, 168, 169, 685, 174, 3761, 5298, 7345, 9397, 5302, 183, 1209, 200, 719, 4815, 4816, 10450, 213, 5846, 6871, 9942, 9943, 218, 6366, 3809, 228, 1261, 2800, 1777, 755, 1779, 5365, 9464, 8450, 2820, 4871, 5389, 784, 6934, 3352, 3354, 283, 284, 795, 796, 6940, 1316, 7461, 12580, 12581, 12583, 12584, 811, 818, 6962, 8499, 12597, 4407, 5431, 5947, 318, 830, 320, 832, 322, 8514, 6469, 3910, 328, 4424, 10056, 9547